# Example #1: Exploring Recent Shootings in Philadelphia with Panel, Altair, and Hvplot

In this notebook, we create an example dashboard visualizing the geographic distribution of recent shootings in Philadelphia. The dashboard includes:

1. An Altair bar chart of the top 20 neighborhoods with the most shootings
1. An hvplot choropleth map showing the total number of shootings per neighborhood
1. A slider for selecting the number of days to query shootings for

#### References

For more information, see the documentation:

- [Documentation homepage](https://panel.holoviz.org)
- [App Gallery](https://panel.holoviz.org/gallery/index.html)
    - Examples of end-to-end web apps using Panel
- [Component Gallery](https://panel.holoviz.org/reference/index.html)
    - Examples (code snippets) for the types of panes, widgets, and layouts possible in Panel dashboards


::: {.callout-note}

For this dashboard, see the **Altair/Vega** [reference gallery page](https://panel.holoviz.org/reference/panes/Vega.html) for more information about embedding Altair in Panel dashboards.
:::

In [1]:
import panel as pn

pn.extension("vega")  # Enables support for vega/altair

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests

import holoviews as hv
import hvplot.pandas

import param as pm

import altair as alt
import folium
from folium.plugins import HeatMap

In [3]:
# Ignore warnings
np.seterr("ignore");

## Load the full data set

We'll load the full data set before initializing the app, and then the user will be able to filter and display subsets of the entire data set. 

The maximum of our slider will be 365 days, so we only need to query data from the past year. When the user slides the slider, we can select the subset of this full dataset to apply.

In [4]:
# The maximum number of days into past to query
# NOTE: this will be the maximum number of days the user will be allowed to select
MAX_DAYS = 365
MIN_DAYS = 30

Query the CARTO database for shootings:

In [5]:
# The CARTO API endpoint
CARTO_URL = "https://phl.carto.com/api/v2/sql"

# Select everything within the last year
# NOTE: "current_date" is a SQL variable built in to the database
WHERE = f"date_ >= current_date - {MAX_DAYS}"

# The query parameters
params = {"q": "SELECT * FROM shootings", "format": "geojson", "where": WHERE}

Make the get request to the API:

In [6]:
# The request
response = requests.get(CARTO_URL, params=params)

# Make the GeoDataFrame
DATA = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326").to_crs(epsg=3857)

Do some formatting:

In [7]:
# Remove entries with missing values
DATA = DATA.dropna(subset=["point_x", "point_y"]).to_crs(epsg=3857)

# Parse the "date_" column into a Datetime object
DATA["date_"] = pd.to_datetime(DATA["date_"].str.slice(0, 10), format="%Y-%m-%d")

Add neighborhood information as well:

In [8]:
# Read in the neighborhood file
URL = "https://raw.githubusercontent.com/MUSA-550-Fall-2023/week-3/main/data/zillow_neighborhoods.geojson"
HOODS = (
    gpd.read_file(URL).rename(columns={"ZillowName": "Neighborhood"}).to_crs(epsg=3857)
)

# Perform a spatial join to add a neighborhood column
DATA = gpd.sjoin(DATA, HOODS, how="left", predicate="within").dropna(
    subset=["Neighborhood"]
)

In [9]:
class ShootingsByNeighborhood(pm.Parameterized):
    """
    An example app visualizing recent shootings in Philadelphia.

    It includes:
    - a slider widget
    - an Altair bar chart
    - an hvplot line chart

    There is a single parameter:

    1. "days": the number of days to query shootings for
    """

    # The number of days to get data for
    days = pm.Integer(default=90, bounds=(MIN_DAYS, MAX_DAYS))

    def get_data(self):
        """
        Return the subset of the full data set ('DATA') that
        occurred in the last 'self.days' days.
        """
        # Today's date
        today = pd.to_datetime("today")

        # Difference between shootings and today
        diff = today - DATA["date_"]

        # Valid selection: less than X days ago
        selection = diff.dt.days < self.days

        # only return subset of data that is necessary
        subset = DATA.loc[selection]

        return subset

    def get_shootings_by_neighborhood(self):
        """
        Return a DataFrame with two columns ('date_', 'count')
        that gives the total number of shootings in a day.
        """
        # data from past X days
        df = self.get_data()

        # Add the neighborhood geometries
        N = HOODS.merge(
            df.groupby("Neighborhood").size().reset_index(name="Shootings"),
            on="Neighborhood",
            how="left",
        )

        # NaN shootings means no shootings occurred
        N["Shootings"] = N["Shootings"].fillna(0)

        return N

    @pm.depends("days")
    def pane_summary_text(self):
        """
        Get a summary of the number of shootings/homicides.

        Returns an HTML <p> tag.
        """
        # only filter this by days
        data = self.get_data()

        # count shootings and homicides
        shootings = len(data)
        t = f"<p style='font-size: 20px'>Showing data for {shootings:,} shootings in the last {self.days} days.</p>"

        return pn.pane.HTML(t, width=500)

    @pm.depends("days")
    def pane_choropleth(self):
        """
        Return an altair histogram of the number of currently selected
        shootings by age.
        """
        # get the filtered data
        data = self.get_shootings_by_neighborhood()

        return data.hvplot.polygons(
            c="Shootings",
            cmap="viridis",
            hover_cols=["Neighborhood", "Shootings"],
            frame_width=400,
            frame_height=375,
            geo=True,
            crs=3857,
        )

    @pm.depends("days")
    def pane_bar_chart(self):
        """
        Return an altair histogram of the number of currently selected
        shootings by age.
        """
        # get the filtered data
        data = self.get_shootings_by_neighborhood()

        # Sort in descending order and select the top 20
        data = data.sort_values(by="Shootings", ascending=False)
        data = data.iloc[:20]

        # create the chart
        chart = (
            alt.Chart(data[["Shootings", "Neighborhood"]])
            .mark_bar()
            .encode(
                x="Shootings",
                y=alt.Y(
                    "Neighborhood:N",
                    sort=alt.EncodingSortField(
                        field="Shootings",  # The field to use for the sort
                        order="descending",  # The order to sort in
                    ),
                ),
                tooltip=["Shootings", "Neighborhood"],
            )
            .properties(width=300, height=500, title="Top 20 Neighborhoods")
        )
        return chart

Initialize the app:

In [10]:
app = ShootingsByNeighborhood(name="")

## Testing your code!

You can test if specific functions from your dashboard by calling a specific function. 

For example, if you call the `pane_bar_chart()` function, you should see your altair chart!

In [11]:
app.pane_bar_chart()

alt.Chart(...)

## Layout our Panel object

We can use a combination of the `Column()` and `Row()` objects in Panel to create the layout in the main component.

::: {.callout-note}

- The `app.param` is an automatically generated set of widgets that corresponds to our `param` parameters
- The charts are specified as the functions of our main application, e.g., app.bar_chart is the function that will return our bar chart.

:::

In [12]:
# The title
title = pn.pane.Markdown("<h1>Recent Shootings in Philadelphia by Neighborhood</h1>", width=1000)

In [13]:
# Layout the dashboard
panel = pn.Column(
    pn.Row(title),
    pn.Row(app.pane_summary_text, app.param),
    pn.Row(app.pane_bar_chart, app.pane_choropleth),
)

### Call servable() and render our Panel object

- The final step is to call the `.servable()` function
- This will render the dashboard directly in the notebook
- It also enables the notebook to be served from `localhost`.

From the command line, we will run:

```
panel serve --show app1.ipynb
```

And see the app live at: `http://localhost:5006/app1`

In [14]:
panel.servable()

Column
    [0] Row
        [0] Markdown(str, width=1000)
    [1] Row
        [0] ParamMethod(method, _pane=HTML, defer_load=False)
        [1] Column(margin=(5, 10))
            [0] StaticText(value='<b></b>')
            [1] IntSlider(end=365, name='Days', start=30, value=90)
    [2] Row
        [0] ParamMethod(method, _pane=Vega, defer_load=False)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)